In [34]:
import shap
import numpy as np
import torch
import matplotlib.pyplot as plt
from monai.networks.nets import DenseNet121

In [ ]:
#config
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# modelo
modelo_path = "./modelo_densenet_256_gradcamConID_cv_resize_small_hu_m300_1400_separadas_bs4_lr0.001_bs4_lr0.001_fold2.pth"

def build_model():
    return DenseNet121(
        spatial_dims=3,
        in_channels=1,
        out_channels=2,
        dropout_prob=0.5
    )

model = build_model()
model.load_state_dict(torch.load(modelo_path, map_location=device))
model.to(device)
model.eval()

print(" Modelo cargado en", device)

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


 Modelo cargado en cuda


In [37]:
from scipy.ndimage import zoom

def upsample_to_64(vol_8):
    """
    Upsamplea un volumen (8,8,8) a (64,64,64) con interpolación lineal.
    """
    return zoom(vol_8, (8,8,8), order=1)

In [ ]:
# funcion de prediccion
def predict_fn(x_numpy):
    batch_size = x_numpy.shape[0]
    upsampled_batch = []
    for i in range(batch_size):
        vol_8 = x_numpy[i].reshape(8,8,8)
        vol_64 = upsample_to_64(vol_8)
        upsampled_batch.append(vol_64)
    
    upsampled_batch = np.stack(upsampled_batch, axis=0)
    x_tensor = torch.tensor(upsampled_batch, dtype=torch.float32).unsqueeze(1).to(device)  # (batch, 1, 64,64,64)
    
    with torch.no_grad():
        logits = model(x_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
    return probs

In [ ]:
# volume
volume_path = "../../volumenes/preprocesados/preprocesamientos_chiquitos/resize_mini_hu_m300_1400_separadas/npy/images/88HASH.npy"
volume = np.load(volume_path)   # (64, 64, 64)
print(" Volumen shape:", volume.shape)

# downsample
print(" Downsampling volumen 64x64x64 -> 8x8x8 (promedio por bloque)")
downsampled = volume.reshape(8,8,8,8,8,8).mean(axis=(1,3,5))
print(" Nuevo shape:", downsampled.shape)

# prep,(samples, features)
input_vector = downsampled.flatten().reshape(1, -1)  # (1, 512)
print(" Input para SHAP:", input_vector.shape)


background = np.zeros_like(input_vector)

#shap explainer
explainer = shap.KernelExplainer(predict_fn, background)
print(" Explainer inicializado")

print(" Calculando SHAP values (puede tardar)...")
shap_values = explainer.shap_values(input_vector, nsamples=100)
print(" SHAP values calculados")

# reshape
print(" Verificando shapes SHAP:")
print("shap_values[0].shape:", shap_values[0].shape)

# extraemos columnas:
shap_values_class0 = shap_values[0][:, 0]
shap_values_class1 = shap_values[0][:, 1]

print(" Shapes por clase:", shap_values_class0.shape, shap_values_class1.shape)

shap_map_class0 = shap_values_class0.reshape(8,8,8)
shap_map_class1 = shap_values_class1.reshape(8,8,8)




 Volumen shape: (64, 64, 64)
 Downsampling volumen 64x64x64 -> 8x8x8 (promedio por bloque)
 Nuevo shape: (8, 8, 8)
 Input para SHAP: (1, 512)
 Explainer inicializado
 Calculando SHAP values (puede tardar)...


  0%|          | 0/1 [00:00<?, ?it/s]

 SHAP values calculados
 Verificando shapes SHAP:
shap_values[0].shape: (512, 2)
 Shapes por clase: (512,) (512,)


In [40]:
slice_idx = 4

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.title("SHAP - Clase 0 (No complicación)")
plt.imshow(shap_map_class0[slice_idx,:,:], cmap='bwr')
plt.colorbar()

plt.subplot(1,2,2)
plt.title("SHAP - Clase 1 (Complicación)")
plt.imshow(shap_map_class1[slice_idx,:,:], cmap='bwr')
plt.colorbar()

plt.tight_layout()
plt.savefig("shap_explanation_downsampled.png")
plt.close()
print(" Imagen SHAP guardada en 'shap_explanation_downsampled.png'")

 Imagen SHAP guardada en 'shap_explanation_downsampled.png'


Hacemos otra prueba:

In [ ]:
import shap
import numpy as np
import torch
import matplotlib.pyplot as plt
from monai.networks.nets import DenseNet121
from scipy.ndimage import zoom
import os

# config
device = "cuda" if torch.cuda.is_available() else "cpu"

modelo_path = "./modelo_densenet_256_gradcamConID_cv_resize_small_hu_m300_1400_separadas_bs4_lr0.001_bs4_lr0.001_fold2.pth"
volume_path = "../../volumenes/preprocesados/preprocesamientos_chiquitos/resize_mini_hu_m300_1400_separadas/npy/images/88HASH.npy"
output_dir = "./shap_slices_output"
os.makedirs(output_dir, exist_ok=True)

# load model
def build_model():
    return DenseNet121(
        spatial_dims=3,
        in_channels=1,
        out_channels=2,
        dropout_prob=0.5
    )

model = build_model()
model.load_state_dict(torch.load(modelo_path, map_location=device))
model.to(device)
model.eval()
print(" Modelo cargado")

#load volume
volume = np.load(volume_path)
print(" Volumen original:", volume.shape)

# Normalize
volume = (volume - volume.min()) / (volume.max() - volume.min() + 1e-8)

#Downsampling
print("Downsampling 64x64x64 -> 16x16x16 con bloques 4x4x4")
downsampled = volume.reshape(16,4,16,4,16,4).mean(axis=(1,3,5))
print("Shape downsampled:", downsampled.shape)

input_vector = downsampled.flatten().reshape(1, -1)
print("Input SHAP:", input_vector.shape)

# upsampling
def upsample_to_64(vol_16):
    return zoom(vol_16, (4,4,4), order=1)

# prediction func
def predict_fn(x_numpy):
    batch_size = x_numpy.shape[0]
    upsampled_batch = []
    for i in range(batch_size):
        vol_16 = x_numpy[i].reshape(16,16,16)
        vol_64 = upsample_to_64(vol_16)
        upsampled_batch.append(vol_64)
    upsampled_batch = np.stack(upsampled_batch, axis=0)
    x_tensor = torch.tensor(upsampled_batch, dtype=torch.float32).unsqueeze(1).to(device)
    with torch.no_grad():
        logits = model(x_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
    return probs

# background
background = np.zeros_like(input_vector)

# explainer
explainer = shap.KernelExplainer(predict_fn, background)
print(" SHAP explainer creado")

print(" Calculando SHAP values (puede tardar)...")
shap_values = explainer.shap_values(input_vector, nsamples=300)
print(" SHAP calculado")

#extremos clases
print(" Shapes SHAP:", shap_values[0].shape)

# SHAP devuelve (512, 2)
shap_values_class0 = shap_values[0][:, 0]
shap_values_class1 = shap_values[0][:, 1]

shap_map_class0 = shap_values_class0.reshape(16,16,16)
shap_map_class1 = shap_values_class1.reshape(16,16,16)

print("SHAP maps reconstruidos (16x16x16)")

#Upsample a 64x64x64
shap_map_class0_up = zoom(shap_map_class0, (4,4,4), order=1)
shap_map_class1_up = zoom(shap_map_class1, (4,4,4), order=1)
print("Upsampled a:", shap_map_class0_up.shape)

# guardamos volumen completo
np.save(os.path.join(output_dir, "shap_class0_volume.npy"), shap_map_class0_up)
np.save(os.path.join(output_dir, "shap_class1_volume.npy"), shap_map_class1_up)
print(" Volúmenes SHAP guardados como .npy")

#guardmaos todas las slices
for z in range(64):
    plt.imsave(
        os.path.join(output_dir, f"class0_slice_{z}.png"),
        shap_map_class0_up[z,:,:],
        cmap='bwr'
    )
    plt.imsave(
        os.path.join(output_dir, f"class1_slice_{z}.png"),
        shap_map_class1_up[z,:,:],
        cmap='bwr'
    )
print(" Todas las slices guardadas como PNG en:", output_dir)


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


 Modelo cargado
 Volumen original: (64, 64, 64)
Downsampling 64x64x64 -> 16x16x16 con bloques 4x4x4
Shape downsampled: (16, 16, 16)
Input SHAP: (1, 4096)
 SHAP explainer creado
 Calculando SHAP values (puede tardar)...


  0%|          | 0/1 [00:00<?, ?it/s]

 SHAP calculado
 Shapes SHAP: (4096, 2)
SHAP maps reconstruidos (16x16x16)
Upsampled a: (64, 64, 64)
 Volúmenes SHAP guardados como .npy
 Todas las slices guardadas como PNG en: ./shap_slices_output
